# Manipulating data for import on PostgreSQL

## 1. Library and data import

In [24]:
import pandas as pd
import json
from ast import literal_eval
from sqlalchemy import create_engine
import os

In [25]:
df_final_act = pd.read_csv('cod_final_act_general_summary.csv')
df_final_act_proposal_summary_1 = pd.read_csv('cod_completed_proposal_general_summary_0_5.csv')
df_final_act_proposal_summary_2 = pd.read_csv('cod_completed_proposal_general_summary_5_10.csv')
df_final_act_proposal_summary_3 = pd.read_csv('cod_completed_proposal_general_summary_10_15.csv')
df_final_act_proposal_summary_4 = pd.read_csv('cod_completed_proposal_general_summary_15_17.csv')

In [26]:
df_proposal_1 = pd.read_csv('cod_proposal_general_summary_0_5.csv')
df_proposal_2 = pd.read_csv('cod_proposal_general_summary_5_10.csv')
df_proposal_3 = pd.read_csv('cod_proposal_general_summary_10_20.csv')
df_proposal_4 = pd.read_csv('cod_proposal_general_summary_20_25.csv')

## 2. Data manipulation

### 2.1 Handle ongoing procedures

In [27]:
df_proposal = pd.concat([df_proposal_1, df_proposal_2, df_proposal_3, df_proposal_4])

In [28]:
df_proposal = df_proposal.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'Unnamed: 0.2'])

In [ ]:
# Adding missing columns that are present in the PostgreSQL database to avoid unmatching columns
df_proposal['resolution_text_2'] = None
df_proposal['proposition_text_2'] = None
df_proposal['text_adopted_others_2'] = None
df_proposal['resolution_text_3'] = None
df_proposal['proposition_text_3'] = None
df_proposal['text_adopted_others_3'] = None
df_proposal['final_act_summary'] = None

In [30]:
df_proposal

,url,reference,title,subjects,key_players,key_events,documentation_gateway,transparency,final_act,is_legislative_proposal,...,final_act_text,final_act_url,proposal_summary,resolution_text_2,proposition_text_2,text_adopted_others_2,resolution_text_3,proposition_text_3,text_adopted_others_3,final_act_summary
0,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0255(COD),Justice programme 2028-2034,7.40.02 Judicial cooperation in civil and comm...,{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '0...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,False,...,NaN,NaN,The European Commission's proposed Justice Pro...,None,None,None,None,None,None,None
1,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0405(COD),Placing on the market of genetically modified ...,"3.10.09.06 Agro-genetics, GMOs, 4.60.02 Consum...",{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,...,NaN,NaN,NaN,None,None,None,None,None,None,None
2,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0540(COD),"Union support for asylum, migration and integr...",7.10 Free movement and integration of third-co...,{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",{'Other Members': [{'Name': [{'text': 'ASENS L...,NaN,True,...,NaN,NaN,The European Commission has proposed a regulat...,None,None,None,None,None,None,None
3,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0550(COD),'AgoraEU' programme 2028–2034,"1.20 Citizen's rights, 4.10.03 Child protectio...",{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,True,...,NaN,NaN,"The proposed 'AgoraEU' programme, spanning fro...",None,None,None,None,None,None,None
4,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0227(COD),Global Europe,"6.20 Common commercial policy in general, 8.70...",{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,...,NaN,NaN,"The ""Global Europe"" legislative proposal, intr...",None,None,None,None,None,None,None
0,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0423(COD),Calculation of emission credits for heavy-duty...,3.20.05 Road transport: passengers and freight...,{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,...,NaN,NaN,NaN,None,None,None,None,None,None,None
1,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0422(COD),NaN,NaN,NaN,{},NaN,NaN,NaN,False,...,NaN,NaN,NaN,None,None,None,None,None,None,None
2,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0385(COD),Amendments to the Critical Raw Materials Act,"3.40 Industrial policy, 3.40.14 Industrial com...",{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '0...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,False,...,NaN,NaN,The European Commission has proposed amendment...,None,None,None,None,None,None,None
3,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0103(COD),Incentivising defence-related investments in t...,"3.40.09 Defence and arms industry, 4.70.01 Str...",{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '2...,"[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,True,...,NaN,NaN,The European Commission has introduced a legis...,None,None,None,None,None,None,None
4,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0258(COD),"Pericles V exchange, assistance and training p...","5.

### 2.2 Handle completed procedures

In [31]:
df_final_act_proposal = pd.concat([df_final_act_proposal_summary_1, df_final_act_proposal_summary_2, df_final_act_proposal_summary_3, df_final_act_proposal_summary_4])

In [32]:
df_final_act = df_final_act.merge(df_final_act_proposal[['reference', 'proposal_summary']], on='reference', how='left')

In [33]:
df_final_act = df_final_act.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'Unnamed: 0.2'])

In [ ]:
# Adding missing columns that are present in the PostgreSQL database to avoid unmatching columns
df_final_act['resolution_text_2'] = None
df_final_act['proposition_text_2'] = None
df_final_act['text_adopted_others_2'] = None
df_final_act['resolution_text_3'] = None
df_final_act['proposition_text_3'] = None
df_final_act['text_adopted_others_3'] = None

### 2.3 Fix problematic columns (not readable as a SQL database) for both ongoing and completed procedures

In [ ]:
def fix_json_format(val):
    if pd.isna(val) or val == "":
        return val
    try:
        # Reads the string as a dictionary
        data = literal_eval(str(val))
        # Returns the string as a standard JSON
        return json.dumps(data)
    except (ValueError, SyntaxError):
        # If the value is not a string, keep it as it already is
        return val

In [ ]:
# Fix problematic columns
columns_to_fix = ['key_players', 'key_events']
for col in columns_to_fix:
    if col in df_final_act.columns:
        df_final_act[col] = df_final_act[col].apply(fix_json_format)

In [ ]:
# Fix problematic columns
columns_to_fix = ['key_players', 'key_events']
for col in columns_to_fix:
    if col in df_proposal.columns:
        df_proposal[col] = df_proposal[col].apply(fix_json_format)

In [ ]:
# Changes order of the columns to match the one that has been input into PostgreSQL
df_final_act = df_final_act.reindex(['url', 'reference', 'title', 'subjects', 'key_players', 'key_events',
       'documentation_gateway', 'transparency', 'final_act',
       'is_legislative_proposal', 'Procedure reference', 'Procedure type',
       'Procedure subtype', 'Legislative instrument', 'Amendments and repeals',
       'Legal basis', 'Stage reached in procedure', 'Committee dossier',
       'Mandatory consultation of other institutions', 'Other legal basis',
       'legislative_proposal_url', 'legislative_proposal_text',
       'decisions_url_list', 'resolution_text_1', 'position_text_1',
       'text_adopted_others_1', 'resolution_text_2', 'proposition_text_2',
       'text_adopted_others_2', 'resolution_text_3', 'proposition_text_3',
       'text_adopted_others_3', 'final_act_text', 'final_act_url', 
       'proposal_summary', 'final_act_summary'], axis=1)

In [ ]:
# Changes order of the columns to match the one that has been input into PostgreSQL
df_proposal = df_proposal.reindex(['url', 'reference', 'title', 'subjects', 'key_players', 'key_events',
       'documentation_gateway', 'transparency', 'final_act',
       'is_legislative_proposal', 'Procedure reference', 'Procedure type',
       'Procedure subtype', 'Legislative instrument', 'Amendments and repeals',
       'Legal basis', 'Stage reached in procedure', 'Committee dossier',
       'Mandatory consultation of other institutions', 'Other legal basis',
       'legislative_proposal_url', 'legislative_proposal_text',
       'decisions_url_list', 'resolution_text_1', 'position_text_1',
       'text_adopted_others_1', 'resolution_text_2', 'proposition_text_2',
       'text_adopted_others_2', 'resolution_text_3', 'proposition_text_3',
       'text_adopted_others_3', 'final_act_text', 'final_act_url', 'proposal_summary', 'final_act_summary'], axis=1)

### 2.4 Concat and save the data

In [42]:
df_all = pd.concat([df_proposal, df_final_act]).reset_index().drop(columns='index')

In [45]:
df_all

,url,reference,title,subjects,key_players,key_events,documentation_gateway,transparency,final_act,is_legislative_proposal,...,resolution_text_2,proposition_text_2,text_adopted_others_2,resolution_text_3,proposition_text_3,text_adopted_others_3,final_act_text,final_act_url,proposal_summary,final_act_summary
0,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0255(COD),Justice programme 2028-2034,7.40.02 Judicial cooperation in civil and comm...,"{""European Parliament"": [{""Joint committee res...","{""Legislative proposal published"": {""Date"": ""0...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,False,...,None,None,None,None,None,None,NaN,NaN,The European Commission's proposed Justice Pro...,None
1,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0405(COD),Placing on the market of genetically modified ...,"3.10.09.06 Agro-genetics, GMOs, 4.60.02 Consum...","{""European Parliament"": [{""Joint committee res...","{""Legislative proposal published"": {""Date"": ""1...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,...,None,None,None,None,None,None,NaN,NaN,NaN,None
2,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0540(COD),"Union support for asylum, migration and integr...",7.10 Free movement and integration of third-co...,"{""European Parliament"": [{""Committee responsib...","{""Legislative proposal published"": {""Date"": ""1...","[{'Institution': 'European Commission', 'Docum...",{'Other Members': [{'Name': [{'text': 'ASENS L...,NaN,True,...,None,None,None,None,None,None,NaN,NaN,The European Commission has proposed a regulat...,None
3,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0550(COD),'AgoraEU' programme 2028–2034,"1.20 Citizen's rights, 4.10.03 Child protectio...","{""European Parliament"": [{""Joint committee res...","{""Legislative proposal published"": {""Date"": ""1...","[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,True,...,None,None,None,None,None,None,NaN,NaN,"The proposed 'AgoraEU' programme, spanning fro...",None
4,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0227(COD),Global Europe,"6.20 Common commercial policy in general, 8.70...","{""European Parliament"": [{""Joint committee res...","{""Legislative proposal published"": {""Date"": ""1...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,...,None,None,None,None,None,None,NaN,NaN,"The ""Global Europe"" legislative proposal, intr...",None
5,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0423(COD),Calculation of emission credits for heavy-duty...,3.20.05 Road transport: passengers and freight...,"{""European Parliament"": [{""Committee responsib...","{""Legislative proposal published"": {""Date"": ""1...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,...,None,None,None,None,None,None,NaN,NaN,NaN,None
6,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0422(COD),NaN,NaN,NaN,{},NaN,NaN,NaN,False,...,None,None,None,None,None,None,NaN,NaN,NaN,None
7,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0385(COD),Amendments to the Critical Raw Materials Act,"3.40 Industrial policy, 3.40.14 Industrial com...","{""European Parliament"": [{""Committee responsib...","{""Legislative proposal published"": {""Date"": ""0...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,False,...,None,None,None,None,None,None,NaN,NaN,The European Commission has proposed amendment...,None
8,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0103(COD),Incentivising defence-related investments in t...,"3.40.09 Defence and arms industry, 4.70.01 Str...","{""European Parliament"": [{""Committee responsib...","{""Legislative proposal published"": {""Date"": ""2...","[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,True,...,None,None,None,None,None,None,NaN,NaN,The European Commission has introduced a legis...,None
9,https://oeil.europarl.eu

In [ ]:
# Save the data
df_final_act.to_csv('completed_proc_clean_2025.csv', index=False, encoding='utf-8')
df_proposal.to_csv('ongoing_proc_clean_2025.csv', index=False, encoding='utf-8')
df_all.to_csv('procedures_2025.csv', index=False, encoding='utf-8')